In [1]:
import urllib.request
import platform
import sys
import os

import time
from re import compile, findall
import webbrowser
from hashlib import md5

import multiprocessing
import subprocess
import parseutility2 as pu
import version
import get_cpu_count

import argparse
from distutils.version import LooseVersion

In [2]:
""" GLOBALS """
localVersion = version.version
osName = ""
bits = ""
urlBase = "http://iotcube.korea.ac.kr/"
urlCheck = urlBase + "getbinaryversion/wf1/"
urlDownload = urlBase + "downloads"

In [3]:
def get_platform():
    global osName
    global bits

    pf = platform.platform()
    bits, _ = platform.architecture()
    if "Windows" in pf:
        osName = "win"
        if "64" in bits:
            bits = "64"
        else:
            bits = "86"
    elif "Linux" in pf:
        osName = "linux"
        if "64" in bits:
            bits = "64"
        else:
            bits = "86"
    else:
        osName = "osx"
        bits = ""

In [4]:
def parseFile_shallow_multi(f):
    functionInstanceList = pu.parseFile_shallow(f, "GUI")
    return (f, functionInstanceList)


def parseFile_deep_multi(f):
    functionInstanceList = pu.parseFile_deep(f, "GUI")
    return (f, functionInstanceList)

In [5]:
def openFolder(self):
    path = os.path.join(os.getcwd(), "hidx")
    if osName == "win":
        subprocess.Popen(
            ["explorer", "/select,", path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
    elif osName == "linux":
        subprocess.Popen(
            ["xdg-open", path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
    elif osName == "osx":
        subprocess.Popen(
            ["open", "-R", path],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )

def generate(directory, asbLevel):
    directory = './files'
    absLevel = 0 # Can also be 1
    progress = 0

    proj = directory.replace('\\', '/').split('/')[-1]
    timeIn = time.time()
    numFile = 0
    numFunc = 0
    numLine = 0

    projDic = {}
    hashFileMap = {}
        
    print(directory)
    
    fileList = pu.loadSource(directory)
    numFile = len(fileList)
    
    print('File list is ', fileList)
    if numFile == 0:
        print("Error: Failed loading source files. - Check if you selected proper directory, or if your project contains .c or .cpp files.")
    else:
        if absLevel == 0:
            func = parseFile_shallow_multi
        else:
            func = parseFile_deep_multi

        cpu_count = get_cpu_count.get_cpu_count()
        print('CPU count is ', cpu_count)
        if cpu_count != 1:
            cpu_count -= 1

#         pool = multiprocessing.Pool(processes=cpu_count)
        
#         for idx, tup in enumerate(pool.imap_unordered(func, fileList)):
        for file in fileList:
            tup = func(file)
            f = tup[0]
            print('F is ', f)
            functionInstanceList = tup[1]
            print('Function Instance List is ', functionInstanceList)
            pathOnly = f.split(proj, 1)[1][1:]
            print('Path Only is ', pathOnly)
#             progress = float(idx + 1) / numFile

            numFunc += len(functionInstanceList)
            print('Num of functions is ', numFunc)
            if len(functionInstanceList) > 0:
                numLine += functionInstanceList[0].parentNumLoc
            print('Num line is ', numLine)
            for f in functionInstanceList:
                f.removeListDup()
                path = f.parentFile
                print('Path is ', path)
                absBody = pu.abstract(f, absLevel)[1]
                print('absBody')
                print(absBody)
                absBody = pu.normalize(absBody)
                print('Normalizes absBody')
                print(absBody)
                funcLen = len(absBody)
                print('func len is ', funcLen)
                if funcLen > 50:
                    hashValue = md5(absBody).hexdigest()
                    print('Hashvalue is ', hashValue)
                    try:
                        projDic[funcLen].append(hashValue)
                    except KeyError:
                        projDic[funcLen] = [hashValue]
                    try:
                        hashFileMap[hashValue].extend([pathOnly, f.funcId])
                    except KeyError:
                        hashFileMap[hashValue] = [pathOnly, f.funcId]
                    print('ProjDic[funcLen] is ', projDic[funcLen])
                    print('HashFileMap[hashValue] is ', hashFileMap[hashValue])
                else:
                    numFunc -= 1  # decrement numFunc by 1 if funclen is under threshold
        try:
            os.mkdir("hidx")
        except:
            pass
        packageInfo = str(localVersion) + ' ' + str(proj) + ' ' + str(numFile) + ' ' + str(numFunc) + ' ' + str(numLine) + '\n'
        with open("hidx/hashmark_" + str(absLevel) + "_" + proj + ".hidx", 'w') as fp:
            fp.write(packageInfo)

            for key in sorted(projDic):
                fp.write(str(key) + '\t')
                for h in list(set(projDic[key])):
                    fp.write(h + '\t')
                fp.write('\n')

            fp.write('\n=====\n')

            for key in sorted(hashFileMap):
                fp.write(str(key) + '\t')
                for f in hashFileMap[key]:
                    fp.write(str(f) + '\t')
                fp.write('\n')

        timeOut = time.time()
    return 0

In [6]:
def main():
    try:
        os.mkdir("hidx")
    except:
        pass
    directory = ''
    absLevel = 0
    
    generate(directory, absLevel)

In [7]:
def resource_path(relative_path):
    """ Get absolute path to resource, works for dev and for PyInstaller """
    try:
        base_path = sys._MEIPASS
    except Exception:
        base_path = os.path.abspath(".")

    return os.path.join(base_path, relative_path)

In [8]:
try:
    # Python 3.4+
    if sys.platform.startswith("win"):
        import multiprocessing.popen_spawn_win32 as forking
    else:
        import multiprocessing.popen_fork as forking
except ImportError:
    import multiprocessing.forking as forking

if sys.platform.startswith("win"):
    # First define a modified version of Popen.
    class _Popen(forking.Popen):
        def __init__(self, *args, **kw):
            if hasattr(sys, 'frozen'):
                # We have to set original _MEIPASS2 value from sys._MEIPASS
                # to get --onefile mode working.
                os.putenv('_MEIPASS2', sys._MEIPASS)
            try:
                super(_Popen, self).__init__(*args, **kw)
            finally:
                if hasattr(sys, 'frozen'):
                    # On some platforms (e.g. AIX) 'os.unsetenv()' is not
                    # available. In those cases we cannot delete the variable
                    # but only set it to the empty string. The bootloader
                    # can handle this case.
                    if hasattr(os, 'unsetenv'):
                        os.unsetenv('_MEIPASS2')
                    else:
                        os.putenv('_MEIPASS2', '')


    # Second override 'Popen' class with our modified version.
    forking.Popen = _Popen

In [9]:
""" EXECUTE """
if __name__ == "__main__":
    multiprocessing.freeze_support()
    main()

./files
file is  test.c
AbsPath is  ./files/test.c
File list is  ['./files/test.c']
CPU count is  4


CalledProcessError: Command 'C:\Users\User1\Documents\GitHub\vuddy\hmark\FuncParser-opt.exe "./files/test.c"' returned non-zero exit status 1.